<a href="https://colab.research.google.com/github/tknr/google_colab_stable_diffusion/blob/main/Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title セットアップ

# ライブラリのインストール
!pip install diffusers==0.2.4 transformers scipy ftfy

# アクセス・トークン設定
Access_Token="********" #@param {type:"string"}

# パイプライン構築
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", use_auth_token=Access_Token)
pipe.to("cuda")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112 kB 32.3 MB/s 
     |████████████████████████████████| 4.7 MB 64.7 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 120 kB 70.5 MB/s 
     |████████████████████████████████| 6.6 MB 61.0 MB/s 


Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/522 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.2.4",
  "feature_extractor": [
    "transformers",
    "CLIPFeatureExtractor"
  ],
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [1]:
#@title 画像生成
import random
import torch

# 生成
prompt = "japanese female dj" #@param {type:"string"}

out_pathes = []
num_images = 2 #@param {type:"slider", min:1, max:10, step:1}


prompts =[ prompt ] * num_images

width = 512 #@param [512,256,128]
height = 512 #@param [512,256,128] 
seed = 0 #@param {type:"slider", min:0, max:65535, step:1}
num_inference_steps=50 #@param {type:"slider", min:1, max:50, step:1}

if seed == 0:
  seed = random.randrange(1,65535)
  print(seed)

generator = torch.Generator("cuda").manual_seed(seed)

result = pipe(prompts,width=width,height=height,num_inference_steps=num_inference_steps,generator=generator)

print(result)

images = result["sample"]

for index,image in enumerate(images):

  # 保存
  sentence = prompt.replace(' ','_')
  out_path = "%s_%dx%d_%d_%d_%d.png" % (sentence,width,height,seed,num_inference_steps,index)
  print(out_path)
  image.save(out_path)
  out_pathes.append(out_path)

  # 表示
  from IPython.display import Image,display
  display(Image(out_path))

44736


NameError: ignored

In [ ]:
#@title 画像のダウンロード
from google.colab import files
for index,out_path in enumerate(out_pathes):
  files.download(out_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>